!pip install sklearn

In [1]:
!pip install gensim

In [28]:
!pip install tensorflow

     |████████████████████████████████| 207.1 MB 22 kB/s  eta 0:00:01   |▊                               | 4.6 MB 4.3 MB/s eta 0:00:47     |█                               | 6.0 MB 4.3 MB/s eta 0:00:47     |█                               | 6.8 MB 4.3 MB/s eta 0:00:48     |██                              | 13.0 MB 2.0 MB/s eta 0:01:36     |██▌                             | 16.5 MB 3.4 MB/s eta 0:00:57     |███▌                            | 22.8 MB 499 kB/s eta 0:06:09     |█████▉                          | 37.5 MB 5.2 MB/s eta 0:00:33     |██████████▎                     | 66.4 MB 1.5 MB/s eta 0:01:33     |███████████▍                    | 73.8 MB 2.3 MB/s eta 0:00:57     |████████████▎                   | 79.2 MB 2.0 MB/s eta 0:01:04     |████████████▎                   | 79.3 MB 2.0 MB/s eta 0:01:04     |████████████▋                   | 81.7 MB 1.5 MB/s eta 0:01:23     |█████████████▎                  | 86.0 MB 2.4 MB/s eta 0:00:52     |██████████████▎                 | 92.1 MB 1.8 

     |████████████████████████████████| 146 kB 2.8 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=6460c2981c206013477de93407f4561bd94c2e6b9aea72f26aa0f17e2d627dd9
  Stored in directory: /Users/andreastettejessen/Library/Caches/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


# Part-of-speech tagging

### Import libraries

In [83]:
import os
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pickle


### Download dataste

In [84]:
import urllib.request  #  download files
import zipfile  #  unzip files

dataset_folder = os.path.join(os.getcwd(), "OriginalDataset")

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip'

dataset_path = os.path.join(dataset_folder, "dependency_treebank.zip")

if not os.path.exists(dataset_path):
    urllib.request.urlretrieve(url, dataset_path)
    print("Successful download")

with zipfile.ZipFile(dataset_path,"r") as zip_ref:
    zip_ref.extractall(dataset_folder)
print("Successful extraction")

Successful extraction


### Split dataset

In [85]:
train_path = './Dataset/train/'# Should we use: os.path.join(os.getcwd(), "OriginalDataset")
val_path = './Dataset/val/'
test_path = './Dataset/test/'
original_path = dataset_folder +'/dependency_treebank/'

In [86]:
if not os.path.exists(train_path):
    print("making directory")
    os.makedirs(train_path)
if not os.path.exists(val_path):
    os.makedirs(val_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

In [87]:
original_dataset = os.listdir(original_path)

In [88]:
original_dataset.sort()
org_train = original_dataset[0:100]
org_val = original_dataset[100:150]
org_test = original_dataset[150:]

In [89]:
for f in org_train:
    os.rename(original_path+f, train_path+f)
for f in org_val:
    os.rename(original_path+f, val_path+f)
for f in org_test:
    os.rename(original_path+f, test_path+f)


### Preprocessing

In [90]:
def lower(lst): # DO WE NEED THIS?
    lowercase_list = [x.lower() for x in lst]
    return lowercase_list

### Create dataframe

In [91]:
def create_dataframe(dataset_path: str):
    dataframe_rows = []
    documents = os.listdir(dataset_path)
    for document in documents:
        path = os.path.join(dataset_path, document)
        with open(path, 'r') as f:
            a = [[x for x in ln.split()] for ln in f]
        a2 = [x for x in a if x != []] # OBS! Removing all empty lines in file so we can make an array
        arr = np.array(a2)
        text = arr[:, 0]
        tagg = arr[:, 1]
        #POStuple =  [tagg[x] for x in range(len(text))] #[(text[x], tagg[x]) for x in range(len(text))]
        document_id = int(document[4:8])
        dataframe_row = {
            "document_id": document_id,
            "text": text,
            "POS": tagg
        }
        dataframe_rows.append(dataframe_row)
    # transform the list of rows in a proper dataframe
    df = pd.DataFrame(dataframe_rows)
    df = df[["document_id", "text", "POS"]]

    return df

In [92]:
df_train = create_dataframe(train_path)
df_val = create_dataframe(val_path)
df_test = create_dataframe(test_path)

df_train

,document_id,text,POS
0,95,"[In, reference, to, your, Oct., 9, page-one, a...","[IN, NN, TO, PRP$, NNP, CD, NN, NN, ``, NNP, N..."
1,37,"[Judging, from, the, Americana, in, Haruki, Mu...","[VBG, IN, DT, NNS, IN, NNP, NNP, POS, ``, DT, ..."
2,66,"[Sir, Peter, Walters, ,, 58-year-old, chairman...","[NNP, NNP, NNP, ,, JJ, NN, IN, NNP, NNP, NNP, ..."
3,52,"[PAPERS, :, Backe, Group, Inc., agreed, to, ac...","[NNS, :, NNP, NNP, NNP, VBD, TO, VB, NNP, NNP,..."
4,3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS..."
...,...,...,...
95,82,"[Criticism, in, the, U.S., over, recent, Japan...","[NNP, IN, DT, NNP, IN, JJ, JJ, NNS, VBZ, VBG, ..."
96,71,"[When, Warren, Winiarski, ,, proprietor, of, S...","[WRB, NNP, NNP, ,, NN, IN, NNP, POS, NNP, NNP,..."
97,20,"[The, U.S., ,, claiming, some, success, in, it...","[DT, NNP, ,, VBG, DT, NN, IN, PRP$, NN, NN, ,,..."
98,14,"[Norman, Ricken, ,, 52, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, CC, NN,..."


### Embed the words using GloVe embedding

embeddings_dict = {}

#### Load glove model

In [93]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(embedding_dimension: int = 50) -> gensim.models.keyedvectors.KeyedVectors:
    download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Word2Vec: 300")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

In [94]:
embedding_dimension = 50

embedding_model = load_embedding_model(embedding_dimension)

#### Get vocabulary

In [95]:
def get_word_list(data_frame, lable):
    big_list = []
    text = data_frame[lable]
    for row in data_frame[lable]:
        big_list += row.tolist()

    word_list = set(big_list)
    return word_list

In [96]:
word_list_train = get_word_list(df_train, "text")
word_list_val = get_word_list(df_val, "text")
word_list_test = get_word_list(df_test, "text")

In [97]:
"""---------------------OBS------------------------
Where shoukld we get tags? Is it ok to assume they are in df_train?
"""
tags_list = get_word_list(df_train, "POS")

#### Find OOV

In [98]:
# Function definition

def check_OOV_terms(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                    word_list):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """
    embedding_vocabulary = set(embedding_model.index_to_key) # Was previously: set(embedding_model.vocab.keys())
    oov = set(word_list).difference(embedding_vocabulary)
    return list(oov)


In [99]:
oov1_terms = check_OOV_terms(embedding_model, word_list_train)
#--------------------------OBS--------------------------------------------#
# To make sure OOV2 does not contain words from OOV1 a check is 
# implemented during the embedding in the word to index function. 
# It might be an idea to include this check in the chekc_OOV_terms as well. 
# Question for the professor?
#--------------------------OBS--------------------------------------------#
oov2_terms = check_OOV_terms(embedding_model, word_list_val)
oov3_terms = check_OOV_terms(embedding_model, word_list_test)


print("Total OOV1 terms: {0} ({1:.2f}%)".format(len(oov1_terms), float(len(oov1_terms)) / len(word_list_train)))
print("Total OOV2 terms: {0} ({1:.2f}%)".format(len(oov2_terms), float(len(oov2_terms)) / len(word_list_train)))
print("Total OOV3 terms: {0} ({1:.2f}%)".format(len(oov3_terms), float(len(oov3_terms)) / len(word_list_train)))


Total OOV1 terms: 2346 (0.29%)
Total OOV2 terms: 1524 (0.19%)
Total OOV3 terms: 957 (0.12%)


### Create word_to_index (for one-hot encoding)

In [100]:
from collections import OrderedDict

def create_word_to_idx(vocabulary):
    word_to_idx = OrderedDict()
    idx_to_word = OrderedDict()
    word_to_idx["PADDING"] = 0
    idx_to_word[0] = "PADDING"
    # Start from one. Index 0 is reserved for padding
    current_idx = 1
    for word in vocabulary:
        if word not in word_to_idx:
            word_to_idx[word] = current_idx
            idx_to_word[current_idx] = word
            current_idx += 1
            
    return word_to_idx, idx_to_word

def update_word_to_idx(old_idx_to_word, old_word_to_idx, new_words):
    word_to_idx = old_word_to_idx.copy()
    idx_to_word = old_idx_to_word.copy()
    current_idx = len(word_to_idx)
    for word in new_words:
        if word not in word_to_idx:
            word_to_idx[word] = current_idx
            idx_to_word[current_idx] = word
            current_idx += 1
            
    return word_to_idx, idx_to_word
    

In [101]:
"""----------------------OBS------------------------------------
Should we also compute word to index for the test set and embed
these words, and then use all of them for training and validation
or should we keep the test set seperate and only compute this for
testing?
----------------------OBS------------------------------------"""
word_to_idx_v1, idx_to_word_v1 = create_word_to_idx(set(embedding_model.index_to_key))
word_to_idx_v2, idx_to_word_v2 = update_word_to_idx(idx_to_word_v1, word_to_idx_v1, oov1_terms)
word_to_idx_v3, idx_to_word_v3 = update_word_to_idx(idx_to_word_v2, word_to_idx_v2, oov2_terms)

tag_to_idx, idx_to_tag = create_word_to_idx(tags_list)

In [102]:
def save_idx_word_list(idx_to_word,word_to_idx, write_path):
    if not os.path.exists(os.path.dirname(write_path)):
        try:
            os.makedirs(os.path.dirname(write_path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    with open(write_path, 'wb') as f:
        pickle.dump(obj={"word_to_idx": words_lexicon, "idx_to_word":idx_to_word} , file=f)

### Create embedding matrix

In [103]:
# OBS Computes the OOV with random embeddings
def create_embedding_matrix(embedding_model, embedding_dimension, word_to_idx):
    embedding_matrix = np.zeros((len(word_to_idx), embedding_dimension), dtype=np.float32)
    for word, idx in word_to_idx.items():
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector
                                
    return embedding_matrix

def expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx, old_embedding_matrix):
    embedding_matrix = np.zeros((len(word_to_idx), embedding_dimension), dtype=np.float32)
    embedding_matrix[0:len(old_embedding_matrix)] = old_embedding_matrix
    
    for word, idx in word_to_idx.items():
        if idx >= len(old_embedding_matrix):
            try:
                embedding_vector = embedding_model[word]
            except (KeyError, TypeError):
                embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)
                
            embedding_matrix[idx] = embedding_vector
    
    return embedding_matrix                                

In [104]:
embedding_matrix_v1 = create_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_v1)
embedding_matrix_v2 = expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_v2, embedding_matrix_v1)
embedding_matrix_v3 = expand_embedding_matrix(embedding_model, embedding_dimension, word_to_idx_v3, embedding_matrix_v2)

print(embedding_matrix_v1.shape)
print(embedding_matrix_v2.shape)
print(embedding_matrix_v3.shape)


(400001, 50)
(402347, 50)
(403291, 50)


#### Test (Remove later)


In [105]:
vocab = ['Hello', 'Solveig', 'is', 'very', 'wierd', '.']

w_t_idx, idx_t_w = create_word_to_idx(vocab)
print(w_t_idx)

embed_matrix = create_embedding_matrix(None, 5, w_t_idx)
print(embed_matrix)

new_words = ['Andrea', 'is', 'super', 'cool', '.']
new_w_t_idx, new_idx_t_w = update_word_to_idx(idx_t_w, w_t_idx, new_words)
print(new_w_t_idx)

new_embed_matrix = expand_embedding_matrix(None, 5, new_w_t_idx, embed_matrix)
print(new_embed_matrix)

OrderedDict([('PADDING', 0), ('Hello', 1), ('Solveig', 2), ('is', 3), ('very', 4), ('wierd', 5), ('.', 6)])
[[ 0.03997208  0.00246665 -0.00069787  0.03213861 -0.04520808]
 [ 0.01709219  0.00129411 -0.01262426  0.0485404   0.01751199]
 [-0.0372045   0.01828186  0.03109608  0.04075774  0.01572894]
 [ 0.03928509  0.01836319  0.03825684  0.02422366 -0.02967703]
 [-0.03588275 -0.02900422  0.02329878 -0.02268077  0.02943369]
 [ 0.02198299  0.04680008  0.02915153 -0.02601662  0.03152778]
 [-0.03792001 -0.01235144  0.02791452  0.01862329 -0.00011678]]
OrderedDict([('PADDING', 0), ('Hello', 1), ('Solveig', 2), ('is', 3), ('very', 4), ('wierd', 5), ('.', 6), ('Andrea', 7), ('super', 8), ('cool', 9)])
[[ 0.03997208  0.00246665 -0.00069787  0.03213861 -0.04520808]
 [ 0.01709219  0.00129411 -0.01262426  0.0485404   0.01751199]
 [-0.0372045   0.01828186  0.03109608  0.04075774  0.01572894]
 [ 0.03928509  0.01836319  0.03825684  0.02422366 -0.02967703]
 [-0.03588275 -0.02900422  0.02329878 -0.0226807

### Transform documents to sequences of integers

In [106]:
def sequence_to_integers(sequence, word_to_idx):
    integers = []
    for element in sequence:
        integers.append(word_to_idx[element])
    return np.array(integers)
"""OBS
Takes only one list of words to integer, not the list with lists.
"""

'OBS\nTakes only one list of words to integer, not the list with lists.\n'

In [107]:
# Make train text and tags into sequence of integers
train_text = df_train["text"]
train_tags = df_train["POS"]

#---------------OBS----------------------------------------------------------------
#  Should x_train be numpy array?
#  Should we use Word to index for V3 or V2???
#---------------OBS----------------------------------------------------------------
x_train = [sequence_to_integers(sequence, word_to_idx_v3) for sequence in train_text]
y_train = [sequence_to_integers(sequence, tag_to_idx) for sequence in train_tags]


val_text = df_val["text"]
val_tags = df_val["POS"]
x_val = [sequence_to_integers(sequence, word_to_idx_v3) for sequence in val_text]
y_val = [sequence_to_integers(sequence, tag_to_idx) for sequence in val_tags]

In [108]:
from keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs, max_seq_len):
    # Keras provides a convenient padding function; 
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return padded_idxs

In [109]:
def find_max_length(seqs):
    max_len = 0
    for seq in seqs:
        if len(seq) > max_len:
            max_len = len(seq)
    return max_len

In [110]:
from keras.utils.np_utils import to_categorical

"""-------------------------OBS--------------------------
Needs to transform y to binary matrix with to_categorical (not so sure exactly why)
----------------------------------------------------------"""

x_train = pad_idx_seqs(x_train, find_max_length(x_train))
y_train = to_categorical(pad_idx_seqs(y_train, find_max_length(y_train)))

x_val = pad_idx_seqs(x_val, find_max_length(x_val))
y_val = to_categorical(pad_idx_seqs(y_val, find_max_length(y_val)))



### Create baseline model

##### two layers architecture: a Bidirectional LSTM layer and a Dense/Fully-Connected layer on top

OBS: this section is not used atm

In [71]:
from tensorflow import keras
from tensorflow.keras import layers


def create_model(layers_info, compile_info) -> keras.Model:
    """
    Create a Keras model given a list of layer information

    :param layers_info: a list of dictionaries, one for each layer
    :param compile_info: dictionary containing compile information

    :return
        model: the built keras sequential model
    """

    print('Found {} total layers'.format(len(layers_info)))

    model = keras.Sequential()
    for info_idx, info in enumerate(layers_info):

        layer = info['layer'](**{key: value for key, value in info.items() if key != 'layer'})
        if info["name"] == "Bidirectional_LSTM_layer":
            model.add(layers.Bidirectional(layers.LSTM(64)))
        else:
            model.add(layer)

    # Debug
    model.summary()

    # Compile
    model.compile(**compile_info)

    return model

In [72]:
baseline_layers_info = [
    {
        "layer": layers.Embedding,
        "output_dim": embedding_dimension,
        "input_dim": len(word_to_idx_v3),
        "weights": [embedding_matrix_v3],
        "input_length": find_max_length(x_train),
        "mask_zero": True, # Because of padding
        "name": "embedding_layer"
    },
    {
        "layer": layers.LSTM,
        "units": 50,
        "name": "Bidirectional_LSTM_layer"
    },
    {
        "layer": layers.Dense,
        "units": len(tag_to_idx),
        "activation": "softmax",
        "name": "Dense_layer"
    }
]

compile_info = {
    'optimizer': keras.optimizers.Adam(learning_rate=1e-3),
    'loss': keras.losses.BinaryCrossentropy(),# Get new error if we change loss to this: 'sparse_categorical_crossentropy',
    'metrics': [keras.metrics.SparseCategoricalAccuracy()],
}

In [73]:
model = create_model(baseline_layers_info, compile_info)

Found 3 total layers
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 2900, 50)         20164550  
                                                                 
 bidirectional_8 (Bidirectio  (None, 2900, 128)        58880     
 nal)                                                            
                                                                 
 Dense_layer (Dense)         (None, 2900, 46)          5934      
                                                                 
Total params: 20,229,364
Trainable params: 20,229,364
Non-trainable params: 0
_________________________________________________________________


### New attempt at buliding baseline

https://www.kaggle.com/tanyadayanand/pos-tagging-using-rnn/notebook

Not sure about the parameters etc.. but at least it works to train the model

In [111]:
from tensorflow import keras
from tensorflow.keras import layers

def create_baseline_model():
    bidirect_model = keras.Sequential()
    bidirect_model.add(layers.Embedding(input_dim = len(word_to_idx_v3),
                                 output_dim    = embedding_dimension,
                                 input_length  = find_max_length(x_train),
                                 weights       = [embedding_matrix_v3],
                                 mask_zero     = True
                                ))
    bidirect_model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    #bidirect_model.add(layers.TimeDistributed(layers.Dense(len(tag_to_idx), activation='softmax')))
    bidirect_model.add(layers.Dense(len(tag_to_idx), activation='softmax'))
    
    return bidirect_model

In [112]:
baseline_model = create_baseline_model()
baseline_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 2900, 50)          20164550  
                                                                 
 bidirectional_10 (Bidirecti  (None, 2900, 128)        58880     
 onal)                                                           
                                                                 
 dense_7 (Dense)             (None, 2900, 46)          5934      
                                                                 
Total params: 20,229,364
Trainable params: 20,229,364
Non-trainable params: 0
_________________________________________________________________


## Train model

In [113]:
baseline_model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              metrics=['acc'])
baseline_model.fit(x=x_train, y=y_train, batch_size=128, epochs=10)

Epoch 1/10
1/1 [==============================] - 34s 34s/step - loss: 0.1146 - acc: 0.0081
Epoch 2/10
1/1 [==============================] - 8s 8s/step - loss: 0.1105 - acc: 0.0091
Epoch 3/10
1/1 [==============================] - 8s 8s/step - loss: 0.1064 - acc: 0.0103
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 0.1025 - acc: 0.0126
Epoch 5/10
1/1 [==============================] - 8s 8s/step - loss: 0.0985 - acc: 0.0156
Epoch 6/10
1/1 [==============================] - 8s 8s/step - loss: 0.0945 - acc: 0.0210
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 0.0904 - acc: 0.0269
Epoch 8/10
1/1 [==============================] - 8s 8s/step - loss: 0.0861 - acc: 0.0330
Epoch 9/10
1/1 [==============================] - 8s 8s/step - loss: 0.0818 - acc: 0.0373
Epoch 10/10
1/1 [==============================] - 7s 7s/step - loss: 0.0774 - acc: 0.0402


In [52]:
y_train.shape

(100, 2900)

In [53]:
x_train.shape

(100, 2900)

In [54]:
embedding_matrix_v3.shape

(403291, 50)